## Exercise 28 - MongoDB

In [1]:
from pymongo import MongoClient 
## pymongo version: # 4.6.1
from pymongo.errors import DuplicateKeyError, CollectionInvalid
import json

In [2]:
class MongoDBConnection:

    @staticmethod
    def read_configuration(file_name):
        with open(file_name, mode= "r") as file:
            config = json.load(file)
        return config
    
    def __init__(self, file_name= "config.json"):
        config = MongoDBConnection.read_configuration(file_name)
        self.collection_name = config["collection_name"]
        self.client = MongoClient(config["host"], config["port"])
        self.db = self.client[config["db_name"]]
        self.setup_collection() # set another attribute in this method


    def setup_collection(self):
        student_schema = {
            "$jsonSchema":{
                "bsonType": "object",
                "required": ["student_id", "name", "age"],
                "properties":{
                    "student_id" : {
                        "bsonType": "string", 
                        "description": "must be string"
                    },
                     "name" : {
                        "bsonType": "string", 
                        "description": "must be string"
                    },
                     "age" : {
                        "bsonType": "integer", 
                        "description": "must be integer"
                    }
                }
            }
        }
        try:
            self.COLLECTION = self.db.create_collection(self.collection_name, validator= student_schema)
        except:
            self.COLLECTION = self.db[self.collection_name]
        self.COLLECTION.create_index("student_id", unique=True)

In [3]:
class Student:
    def __init__(self, student_id, name, age, **attributes):
        self.student_id = student_id
        self.name = name
        self.age = age
        self.attributes = attributes


    def change_to_dict(self):
        student = {
            "student_id": self.student_id,
            "name": self.name,
            "age": self.age,
            **self.attributes
        }
        return student
    

In [4]:
class Management:
    def __init__(self, db_connection: MongoDBConnection):
        self.db_connection = db_connection

    def add_student(self, s_id):
        ## Check if the student id exists
        std = self.db_connection.COLLECTION.find_one({"student_id": s_id})
        if std:
            print(f"There is already a student associated with the ID number: {s_id}")
            return
        ## Acquire name and age from the user
        name = input("Please provide a value for the `name` : ")
        while True:
            age = input("Please provide a value for the `age` : ")
            try:
                age = int(age)
                break
            except Exception as e:
                print("`age` must be an integer number!")
        ## Acquire additional information from the user 
        additional_attrs = {}
        immutable_fields = ["_id", "student_id"] # cannot be modified
        while True:
            attr_k = input("Please enter a label for the new information you would like to provide. or press Enter to skip: ")
            if not attr_k or attr_k in immutable_fields:
                break
            attr_v = input(f"Please provide the value for `{attr_k}` or press Enter to skip: ")
            if not attr_v:
                break
            additional_attrs[attr_k] = attr_v
        ## Create a dictionary
        student = Student(s_id, name, age, **additional_attrs)
        student = student.change_to_dict()
        ## Execute MongoDB command
        try:
            insert_result = self.db_connection.COLLECTION.insert_one(student)
            if insert_result.inserted_id:
                print("The student's record has been successfully added!")
            else:
                print("The student's record cannot be added!")
        except DuplicateKeyError as e:
            print(f"Error: The ID already exists!")
        except CollectionInvalid as e:
            print("Error: Collection not found!")


    def search_student(self, s_id):
        std = self.db_connection.COLLECTION.find_one({"student_id": s_id})
        if not std:
            print(f"There is no student associated with the ID number: {s_id}")
            return
        print("-"*10, " Student's Profile ", "-"*10)
        info = []
        for k, v in std.items():
            if k != "_id":
                info.append(f"{k}: {v}")
        print(", ".join(info))


    def edit_student(self, s_id):
        ## Check if the student id exists
        std = self.db_connection.COLLECTION.find_one({"student_id": s_id})
        if not std:
            print(f"There is no student associated with the ID number: {s_id}")
            return
        ## Show current profile
        print("-"*10, " Current Profile ", "-"*10)
        info = []
        for k, v in std.items():
            if k != "_id":
                info.append(f"{k}: {v}")
        print(", ".join(info))
        ## Update required fields
        required_fields = ["name", "age"] 
        immutable_fields = ["_id", "student_id"] # cannot be modified
        updated_fields = {}
        for k,v in std.items():
            if k in required_fields:
                new_value = input(f"Please provide a new value for the `{k}` field, or press Enter to keep the previous value: ")
                if new_value:
                    if k == "age":
                        try:
                            new_value = int(new_value)
                            updated_fields[k] = new_value
                        except Exception as e:
                            print("Cannot update `age`: invalid literal")
                    else:
                        updated_fields[k] = new_value
        ## Update optional fields: modify a value or delete a field
        deleted_fields = {}
        for k, v in std.items():
            if k not in required_fields and k not in immutable_fields:
                new_value = input(f"To update the `{k}` field, please enter a new value. Press `Enter` to keep the previous value, or press `d` to remove this field: ")
                if new_value == 'd':
                    deleted_fields[k] = ""
                elif new_value: 
                    updated_fields[k] = new_value
        ## Update optional fields: add new field(s)
        while True:
            k = input("Please enter a label for the new information you would like to provide. or press Enter to skip: ")
            if not k or k in immutable_fields:
                break
            v = input(f"Please provide the value for `{k}` or press Enter to skip: ")
            if not v:
                break
            updated_fields[k] = v
        ## Execute MongoDB commands
        if deleted_fields:  
            update_result = self.db_connection.COLLECTION.update_one({"student_id": s_id},{"$unset": deleted_fields})
            if update_result.modified_count == 0:
                print("Cannot remove the optional fields")
        if updated_fields:
            update_result = self.db_connection.COLLECTION.update_one({"student_id": s_id},{"$set": updated_fields})
            if update_result.modified_count == 0:
                print("Cannot update the requested fields")
            else :
                print("The student's profile is successfully updated!")


    def remove_student(self, s_id):
        delete_result = self.db_connection.COLLECTION.delete_one({"student_id": s_id})
        if delete_result.deleted_count:
            print("The student's record has been successfully removed!")
        else:
            print("No student matching the given ID is found!")


    def view_students(self):
        stds = self.db_connection.COLLECTION.find()
        stds = list(stds)
        if not stds:
            print("There are no students listed in the database!")
            return
        print("-"*10, " Students ", "-"*10)
        for std in stds:
            info = []
            for k, v in std.items():
                if k != "_id":
                    info.append(f"{k}: {v}")
            print(", ".join(info))

In [5]:
def main():
    manager = Management(MongoDBConnection())
    while True:
        print("\n1. Add Student\n2. Search Student\n3. Edit Student\n4. Remove Student\n5. View All Students\n6. Exit")
        choice = input("Choose an option: ")

        ## Add Student
        if choice == '1':
            student_id = input("student id (to add): ")
            manager.add_student(student_id)
                    
        ## Search Student
        elif choice == '2':
            student_id = input("student id (to search): ")
            manager.search_student(student_id)

        ## Edit Student
        elif choice == '3':
            student_id = input("student id (to edit): ")
            manager.edit_student(student_id)

        ## Remove Student
        elif choice == '4':
            student_id = input("student id (to remove): ")
            manager.remove_student(student_id)

        ## View All Students
        elif choice == '5':
            manager.view_students()
        
        ## Exit
        elif choice == '6':
            break


if __name__ == "__main__":
    main()


1. Add Student
2. Search Student
3. Edit Student
4. Remove Student
5. View All Students
6. Exit
Choose an option: 5
----------  Students  ----------
student_id: 2, name: n2, age: 15
student_id: 3, name: n3, age: 54
student_id: 1, name: n1, age: 56, email: rew@yahoo.com, address: ISF
student_id: 4, name: n4, age: 33, address: TEH

1. Add Student
2. Search Student
3. Edit Student
4. Remove Student
5. View All Students
6. Exit
Choose an option: 3
student id (to edit): 1
----------  Current Profile  ----------
student_id: 1, name: n1, age: 56, email: rew@yahoo.com, address: ISF
Please provide a new value for the `name` field, or press Enter to keep the previous value: 
Please provide a new value for the `age` field, or press Enter to keep the previous value: fg
Cannot update `age`: invalid literal
To update the `email` field, please enter a new value. Press `Enter` to keep the previous value, or press `d` to remove this field: d
To update the `address` field, please enter a new value. Pre

## Exercise 29 - Numpy

In [6]:
import numpy as np

Create a 2D NumPy array with dimensions 5x5, filled with integers from 1 to 25.

Use slicing to:

* Extract and display the third row of the array.
* Extract and display the second column of the array.

In [7]:
## Version I: Using Numpy's `array` method
arr_2d = np.array([[12, 20, 10, 8, 23],
                  [1, 21, 17, 24, 13],
                  [13, 10, 7, 3, 18],
                  [7, 8, 9, 20, 15],
                  [12, 6, 1, 24, 5]])
print(f"2D NumPy array:\n{arr_2d}")
print(f"The shape of the NumPy array: {arr_2d.shape}")
print(f"The data type of the NumPy array: {arr_2d.dtype}")

# ## Version II: Using Numpy's `randint` method
# arr_2d = np.random.randint(low= 1, high= 26, size=(5,5))
# print(f"2D NumPy array:\n{arr_2d}")
# print(f"The shape of the NumPy array: {arr_2d.shape}")

## Slicing
print("-"*20, " Slicing ", "-"*20)
third_row = arr_2d[2,:]
print(f"The third row: {third_row}")
print(f"The shape of the third row: {third_row.shape}")

second_col = arr_2d[:,1]
print(f"The second coloumn: {second_col}")
print(f"The shape of the second coloumn: {second_col.shape}")


2D NumPy array:
[[12 20 10  8 23]
 [ 1 21 17 24 13]
 [13 10  7  3 18]
 [ 7  8  9 20 15]
 [12  6  1 24  5]]
The shape of the NumPy array: (5, 5)
The data type of the NumPy array: int32
--------------------  Slicing  --------------------
The third row: [13 10  7  3 18]
The shape of the third row: (5,)
The second coloumn: [20 21 10  8  6]
The shape of the second coloumn: (5,)


Create a 2D NumPyarray with dimensions 3x4, filled with any integers of your choice. Access and print the element at the second row and third column.

In [8]:
arr_2d = np.array([[1, 2, 3, 4],
                   [5, 6, 7, 8],
                   [9, 10, 11, 12]])
element_23 = arr_2d[1,2]
print(f"The element at the second row and third column: {element_23}")
print(f"The data type of the NumPy array: {arr_2d.dtype}")

The element at the second row and third column: 7
The data type of the NumPy array: int32


Create two NumPy arrays of the same size with random integers. Calculate their element-wise sum and then display elements from new arrays wherever the sum is greater than 50.

In [9]:
ndarr_1 = np.random.randint(low=1, high=60, size=(4,4))
ndarr_2 = np.random.randint(low=1, high=60, size=(4,4))
print(f"array1 :\n{ndarr_1}")
print(f"\narray2 :\n{ndarr_2}")
## Element-wise summation
ndarr_sum = ndarr_1 + ndarr_2
print(f"\nSum:\n{ndarr_sum}")

ndarr_sum_cond = ndarr_sum[ndarr_sum > 50]
print(f"\nElements greater than 50: {ndarr_sum_cond}")
print(f"Shape: {ndarr_sum_cond.shape}")


array1 :
[[ 8 15 29 54]
 [12 14 37 47]
 [ 7 50 54 37]
 [49 55  5 24]]

array2 :
[[58 29  1  2]
 [19 59 40 11]
 [ 5 15 21  8]
 [34 32 17 52]]

Sum:
[[66 44 30 56]
 [31 73 77 58]
 [12 65 75 45]
 [83 87 22 76]]

Elements greater than 50: [66 56 73 77 58 65 75 83 87 76]
Shape: (10,)
